# Clasificador K-NN en Spark usando pyspark.dataframes

### Se importan las librerías necesarias

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import random
import math
import sys
import numpy as np 

### Se crea la sesión y config. de Spark

In [9]:
conf = (SparkConf()
        .setAppName("Data exploration URL - KNN Spark RDD") \
        .set('spark.driver.cores', '5') \
        .set('spark.executor.cores', '5') \
        .set('spark.driver.memory', '5G') \
        .set('spark.executor.memory', '5G'))
sc = SparkContext(conf=conf)

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [11]:
sc._conf.getAll()

[('spark.driver.port', '35969'),
 ('spark.executor.cores', '5'),
 ('spark.executor.memory', '5G'),
 ('spark.app.id', 'local-1618589946041'),
 ('spark.driver.cores', '5'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '5G'),
 ('spark.app.startTime', '1618589942839'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'Data exploration URL - KNN Spark RDD'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'fedora')]

In [12]:
sc

<SparkContext master=local[*] appName=Data exploration URL - KNN Spark RDD>

In [13]:
import time
def tiempo(start, end):
    medida = ' segundos'
    tiempo = end - start
    if (tiempo >= 60):
        tiempo = tiempo / 60
        medida = ' minutos'
    print("Tiempo de ejecución: ", tiempo)

### Calcular la distancia euclideana.
#### Summary:
        Se calcula la distancia entre las columnas de dos renglones de un dataset, funciona
        con argumentos provenientes de un renglón de un dataframe de Spark.
#### Args: 
        row1(numpy.ndarray): Recibe una instancia del dataset
        row2(pyspark.ml.linalg.SparseVector): Recibe una instancia del dataset

In [14]:
def euclidean_distance(row1, row2):
    distance = 0.0
    columns = len(row1[0])
    for column in range(columns):
        distance += pow(row1[0][column] - row2[column], 2)
    distance = math.sqrt(distance)
    return distance

### Obtener los vecinos más cercanos.
#### Summary: 
      Se recorre cada renglón del dataframe dado y se calcula la distancia entre cada 
      uno de estos y el renglón de prueba.
      El RDD "distances", almacenará las distancias calculadas, 
      posteriormente se ordena de modo ascendente y se almancenan los primeros k-elementos 
      en la lista "k_neighbors"

#### Args: 
      train(pyspark.rdd.RDD): Recibe el conjunto de entrenamiento
      test_row(numpy.ndarray): Recibe una instancia del conjunto de test
      k(int): Número de vecinos que se desean obtener

In [15]:
def get_neighbors(train, test_row, k):
    rdd_distances = train.map(lambda element: (element[0], euclidean_distance(test_row, element[1])))
    rdd_distances = rdd_distances.filter(lambda element: element[1] > 0.0)
    k_neighbors = rdd_distances.takeOrdered(k, key= lambda  x: x[1])
    return k_neighbors

### Predecir las etiquetas usando k-nn.
#### Summary:
      Se obtiene la lista de los k-vecinos más cercanos, y se almacena el valor de
      la etiqueta en la lista "output_labels". Posteriormente se calcula el valor 
      promedio de las etiquetas y se almacena en la variable "prediction" y se retorna.

#### Args: 
      train(pyspark.rdd.RDD): Recibe el conjunto de entrenamiento
      test_row(numpy.ndarray): Recibe una instancia del conjunto de test
      k(int): Número de vecinos que se desean obtener

In [16]:
def predict_classification(train, test_row, k):
    neighbors = get_neighbors(train, test_row, k)
    output_labels = [row[0] for row in neighbors]
    prediction = max(set(output_labels), key=output_labels.count)
    return prediction

### Clacular el porcentaje de exactitud.
#### Summary:
      Esta función calcula el porcentaje de exactitud del uso de k-NN, comparando
      las etiquetas reales de las instancias del dataset de entrenamiento y las
      etiquetas obtenidas mediante la predicción usando k-NN.
#### Args: 
      real_labels(numpy.ndarray): Recibe el dataframe de test que contiene los
                                                    valores reales de las etiquetas
      predicted(list): Lista con las etiquetas obtenidas mediante K-NN

In [17]:
def accuracy(real_labels, predicted):
    correct = 0
    total_rows = len(real_labels)
    for i in range(total_rows):
        if(real_labels[i] == predicted[i]):
            correct += 1
    print("Correct labels: ", correct, 'of', (total_rows))
    accuracy = correct / float(total_rows)
    return accuracy

### Crear la función que calcule los vecinos más cercanos.
#### Summary:
      Se asignan los parámetros para calcular los k-vecinos más cercanos y hacer predicciones
      de las etiquetas a las que pertenecen, calculando la distancia entre las columnas de cada
      uno de los renglones del dataframe de "test" y el de "train", comparando las 
      reales con las otenidas por el clasificador y, finalmente, dado el porcentaje de exactitud obtenido. 
#### Args: 
      train(pyspark.rdd.RDD): Recibe el conjunto de entrenamiento
      test(pyspark.rdd.RDD): Recibe el conjunto de test
      k(int): Número de vecinos que se desean obtener

In [18]:
def k_nearest_neighbors(train, test, k):
    predictions = []
    total_test_rows = test.count()
    for index in range(total_test_rows):
        test_row = np.array(test.zipWithIndex().filter(lambda element: element[1] == index).map(lambda element: element[0][1]).collect(), dtype = object)
        output = predict_classification(train, test_row, k)
        predictions.append(output)
    labels_array = np.array(test.map(lambda x: x[0]).collect(), dtype = float)
    mean_accuracy = accuracy(labels_array, predictions)
    print("Mean accuracy: " + str(mean_accuracy))

### Se cargan los datos al dataframe 

In [54]:
# Load training data
data = spark.read.format("libsvm")\
    .option("header", "false")\
    .option("inferSchema","true")\
    .load("../data/url_svmlight/Dimension_500_x_1000.svm")

### Normalización

In [55]:
from pyspark.ml.feature import Normalizer

In [56]:
data_norm = Normalizer(inputCol="features", outputCol="features_norm", p=1).transform(data)

In [57]:
data_norm = data_norm.drop('features')

In [58]:
data.head(1)

[Row(label=1.0, features=SparseVector(997, {3: 0.0581, 4: 0.0966, 5: 0.1176, 15: 0.1, 16: 0.7756, 17: 0.8039, 18: 0.2069, 22: 0.1429, 119: 0.3333, 125: 0.5, 130: 1.0, 131: 0.5, 132: 1.0, 133: 0.5, 135: 0.5, 137: 0.5, 138: 1.0, 140: 1.0, 142: 1.0, 144: 1.0, 146: 1.0, 147: 0.5, 148: 1.0, 269: 0.1429, 667: 1.0}))]

In [59]:
data_norm.head(1)

[Row(label=1.0, features_norm=SparseVector(997, {3: 0.0039, 4: 0.0065, 5: 0.008, 15: 0.0068, 16: 0.0525, 17: 0.0544, 18: 0.014, 22: 0.0097, 119: 0.0226, 125: 0.0338, 130: 0.0677, 131: 0.0338, 132: 0.0677, 133: 0.0338, 135: 0.0338, 137: 0.0338, 138: 0.0677, 140: 0.0677, 142: 0.0677, 144: 0.0677, 146: 0.0677, 147: 0.0338, 148: 0.0677, 269: 0.0097, 667: 0.0677}))]

In [60]:
#Dividir los datos en conjunto de train y de test
seed = 1234
splits = data_norm.randomSplit([0.7, 0.3], seed)

train = splits[0]
test = splits[1]

# Se asignan los RDD para el posterior procesamiento
rdd_train = train.rdd
rdd_test = test.rdd
# rdd_total = data.rdd

### Se llama al método y se envían los parámetros

In [ ]:
start_time = time.time()
k_nearest_neighbors(rdd_train, rdd_test, k = 1)
end_time = time.time()
print(tiempo(start_time, end_time))

## Prueba de cada método de K-NN  con el archivo Dimensión 5 x 76

In [25]:
# Se asignan los RDD para el posterior procesamiento
rdd_train = train.rdd
rdd_test = test.rdd
rdd_total = data.rdd

In [26]:
# Se agrega un índice a las instancias para poder recorrerlas posteriormente mediante un filtro.
rdd_index = rdd_total.zipWithIndex()
# Se selecciona solo la columna que contiene los valores de las características.
rdd_columns = rdd_total.map(lambda x: x[1])

In [164]:
row1[0][75]

1.0

In [28]:
# Se prueba el método de distancia euclideana con RDD
# Renglón no. 1
rdd_row1 = rdd_index.filter(lambda x: x[1] == 0)
# Se transforma en un array de Numpy solo con los valores de las columnas
row1 = np.array(rdd_row1.map(lambda element: element[0][1]).collect(), dtype = object)
# Las distancias se almacenan en un RDD
rdd_distances = rdd_columns.map(lambda x: euclidean_distance(row1, x))
start_time = time.time()
rdd_distances.collect()
end_time = time.time()
print(tiempo(start_time,end_time))

Tiempo de ejecución:  0.0947568416595459
None


In [96]:
# Prueba de la función get_neighbors()
# Renglón no. 1
rdd_row1 = rdd_index.filter(lambda x: x[1] == 0)
# Se transforma en un array de Numpy solo con los valores de las columnas
row1 = np.array(rdd_row1.map(lambda element: element[0][1]).collect(), dtype = object)
start_time = time.time()
print(get_neighbors(rdd_total, row1, 3))
end_time = time.time()
print(tiempo(start_time,end_time))

[(0.0, 1.4404646590152195), (0.0, 1.7914667218708056), (0.0, 2.0356611659021397)]
Tiempo de ejecución:  0.03345060348510742
None


### Prueba actual

In [120]:
# Prueba de la función predict_classification()
# Renglón no. 1
rdd_row1 = rdd_index.filter(lambda x: x[1] == 0)
# Se transforma en un array de Numpy solo con los valores de las columnas
row1 = np.array(rdd_row1.map(lambda element: element[0][1]).collect(), dtype = object)
start_time = time.time()
prediction = predict_classification(rdd_total, row1, 3)
print('Expected label: %d, Got: %d.' % (rdd_row1.take(1)[0][0][0], prediction))
end_time = time.time()
print(tiempo(start_time,end_time))

Expected label: 0, Got: 0.
Tiempo de ejecución:  0.6830856800079346
None


# Segmento de pruebas locas

In [8]:
# Build the SparkSession
# spark = SparkSession.builder \
#    .master("local[6]") \
#    .appName("Data exploration URL - KNN Spark RDD") \
#    .config("spark.executor.memory", "4gb") \
#    .getOrCreate()

# sc = spark.sparkContext

In [229]:
# Load training data
data = spark.read.format("libsvm")\
    .option("header", "false")\
    .option("inferSchema","true")\
    .load("/home/jsarabia/Documents/IA/Data-exploration-url_svmlight/data/url_svmlight/Dimension_100_x_50000.svm")
# Split the data into train and tes100
seed = random.randrange(500, 1300, 2)
splits = data.randomSplit([0.7, 0.3], 1234)

train = splits[0]
test = splits[1]

# Se asignan los RDD para el posterior procesamiento
rdd_train = train.rdd
rdd_test = test.rdd
# rdd_total = data.rdd

In [230]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [231]:
from pyspark.ml.linalg import SparseVector
from pyspark.ml.feature import Normalizer

In [232]:
df_new = Normalizer(inputCol="features", outputCol="features_norm", p=1).transform(data)

In [233]:
df_new.head(1)

[Row(label=0.0, features=SparseVector(49415, {3: 0.0373, 4: 0.0552, 5: 0.1176, 10: 0.2857, 20: 0.2857, 21: 0.0005, 22: 0.001, 27: 1.0, 32: 0.1111, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 91: 1.0, 93: 1.0, 101: 1.0, 103: 1.0, 105: 1.0, 107: 1.0, 111: 1.0, 154: 1.0, 174: 1.0, 179: 1.0, 184: 1.0, 187: 1.0, 188: 1.0, 189: 1.0, 193: 0.001, 385: 1.0, 487: 1.0, 490: 1.0, 577: 1.0, 581: 1.0, 2853: 1.0, 2854: 1.0, 2855: 1.0, 2856: 1.0}), features_norm=SparseVector(49415, {3: 0.001, 4: 0.0015, 5: 0.0031, 10: 0.0075, 20: 0.0075, 21: 0.0, 22: 0.0, 27: 0.0264, 32: 0.0029, 53: 0.0264, 55: 0.0264, 61: 0.0264, 63: 0.0264, 65: 0.0264, 67: 0.0264, 69: 0.0264, 71: 0.0264, 73: 0.0264, 81: 0.0264, 83: 0.0264, 85: 0.0264, 87: 0.0264, 91: 0.0264, 93: 0.0264, 101: 0.0264, 103: 0.0264, 105: 0.0264, 107: 0.0264, 111: 0.0264, 154: 0.0264, 174: 0.0264, 179: 0.0264, 184: 0.0264, 187: 0.0264, 188: 0.0264, 189: 0.0264, 193: 0.0, 385: 0.0264

In [234]:
df_new2 = df_new

In [235]:
df_new2 = df_new2.drop('features')

In [236]:
df_new2.head(1)

[Row(label=0.0, features_norm=SparseVector(49415, {3: 0.001, 4: 0.0015, 5: 0.0031, 10: 0.0075, 20: 0.0075, 21: 0.0, 22: 0.0, 27: 0.0264, 32: 0.0029, 53: 0.0264, 55: 0.0264, 61: 0.0264, 63: 0.0264, 65: 0.0264, 67: 0.0264, 69: 0.0264, 71: 0.0264, 73: 0.0264, 81: 0.0264, 83: 0.0264, 85: 0.0264, 87: 0.0264, 91: 0.0264, 93: 0.0264, 101: 0.0264, 103: 0.0264, 105: 0.0264, 107: 0.0264, 111: 0.0264, 154: 0.0264, 174: 0.0264, 179: 0.0264, 184: 0.0264, 187: 0.0264, 188: 0.0264, 189: 0.0264, 193: 0.0, 385: 0.0264, 487: 0.0264, 490: 0.0264, 577: 0.0264, 581: 0.0264, 2853: 0.0264, 2854: 0.0264, 2855: 0.0264, 2856: 0.0264}))]

In [237]:
seed = random.randrange(500, 1300, 2)
splits = df_new2.randomSplit([0.7, 0.3], 1234)

train = splits[0]
test = splits[1]

# Se asignan los RDD para el posterior procesamiento
rdd_train = train.rdd
rdd_test = test.rdd
# rdd_total = data.rdd

In [238]:
start_time = time.time()
k_nearest_neighbors(rdd_train, rdd_test, k = 1)
end_time = time.time()
print(tiempo(start_time, end_time))

Correct labels:  56 of 69
Mean accuracy: 0.8115942028985508
Tiempo de ejecución:  21.329047699769337
None


In [66]:
temp_data.select('ss_features').show(2)

+--------------------+
|         ss_features|
+--------------------+
|(987,[1,3,4,5,9,1...|
|(987,[3,4,5,10,20...|
+--------------------+
only showing top 2 rows



In [189]:
rdd_row1 = rdd_index.filter(lambda x: x[1] == 0)
rdd_row2 = rdd_index.filter(lambda x: x[1] == 2)

In [178]:
labels_array = np.array(rdd_test.map(lambda x: x[0]).collect(), dtype = float)
labels_array[0]

0.0

In [44]:
start_time = time.time()
euclidean_distance(rdd_row1, rdd_row2)
end_time = time.time()
print(tiempo(start_time,end_time))

Tiempo de ejecución:  31.2671480178833
None


In [88]:
start_time = time.time()
instancias = rdd_total.count()
rdd_row1 = rdd_index.filter(lambda x: x[1] == 0)
row1 = np.array(rdd_row1.map(lambda element: element[0][1]).collect(), dtype = object)
for i in range(instancias):
    rdd_row2 = rdd_index.filter(lambda x: x[1] == i)
    row2 = np.array(rdd_row2.map(lambda element: element[0][1]).collect(), dtype = object)
    distance = euclidean_distance(rdd_row1, rdd_row2)
    # print("Dinstancia del renglon 1 con el", i, ":", distance) 
end_time = time.time()
print(tiempo(start_time,end_time))

Tiempo de ejecución:  18.672855377197266
None


In [ ]:
start_time = time.time()
length = len(total_array)
for row in range(length):
    distance = euclidean_distance(total_array[0], total_array[row])
    #print("Dinstancia del renglon 1 con el", row, ":", distance)    
end_time = time.time()
print(tiempo(start_time,end_time))

In [300]:
# Se utilizan dos array de numpy para alamacenar las instancias del set de entrenamiento y procesar con un RDD y el segundo array almacena las etiquetas. 
#train_array = np.array(train.select('features').collect(), dtype=float)
#train_array_labels = np.array(train.select('label').collect(), dtype=float)

In [301]:
# Etiquteas de las instancias del conjunto de test. 
train_array = np.array(train.collect(), dtype=object)
test_array = np.array(test.collect(), dtype=object)
total_array = np.array(data.collect(), dtype=object)

In [302]:
total_array[0]

array([0.0,
       SparseVector(494785, {3: 0.0373, 4: 0.0552, 5: 0.1176, 10: 0.2857, 20: 0.2857, 21: 0.0005, 22: 0.001, 27: 1.0, 32: 0.1111, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 91: 1.0, 93: 1.0, 101: 1.0, 103: 1.0, 105: 1.0, 107: 1.0, 111: 1.0, 154: 1.0, 174: 1.0, 179: 1.0, 184: 1.0, 187: 1.0, 188: 1.0, 189: 1.0, 193: 0.001, 385: 1.0, 487: 1.0, 490: 1.0, 577: 1.0, 581: 1.0, 2853: 1.0, 2854: 1.0, 2855: 1.0, 2856: 1.0, 51675: 1.0, 51676: 1.0, 51677: 1.0, 51683: 1.0, 64598: 1.0, 106458: 1.0, 155152: 1.0, 155153: 1.0, 155154: 1.0, 155155: 1.0, 155156: 1.0, 155161: 1.0, 155162: 1.0, 155163: 1.0, 155164: 1.0, 155165: 1.0, 155167: 1.0, 155168: 1.0, 155169: 1.0, 155171: 1.0, 155172: 1.0, 155173: 1.0, 155174: 1.0, 155175: 1.0, 155176: 1.0, 155177: 1.0, 155178: 1.0, 155179: 1.0, 155180: 1.0, 155181: 1.0, 155182: 1.0, 155193: 1.0, 155194: 1.0, 155195: 1.0, 155196: 1.0, 155197: 1.0, 155198: 1.0, 155199: 1.0, 155200: 

In [287]:
start_time = time.time()
length = len(total_array)
for row in range(length):
    distance = euclidean_distance(total_array[0], total_array[row])
    #print("Dinstancia del renglon 1 con el", row, ":", distance)    
end_time = time.time()
print(tiempo(start_time,end_time))

Tiempo de ejecución:  0.18266701698303223
None


In [288]:
# Prueba de la función get_neighbors(), se envían como args. el dataframe, un renglón de este y el número de vecinos.
start_time = time.time()
length = data.count() + 1
neighbors = get_neighbors(total_array, total_array[0], k=3)
for neighbor in neighbors:
    print(neighbor)
end_time = time.time()
print(tiempo(start_time, end_time))

[0.0
 SparseVector(100, {3: 0.1037, 4: 0.1655, 5: 0.1176, 10: 0.2857, 20: 0.0238, 21: 0.0238, 27: 1.0, 43: 1.0, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 89: 1.0, 91: 1.0, 93: 1.0, 95: 1.0})]
[0.0
 SparseVector(100, {3: 0.083, 4: 0.131, 5: 0.1176, 10: 0.2857, 20: 0.006, 21: 0.006, 27: 1.0, 32: 0.0556, 40: 0.1, 43: 1.0, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 89: 1.0, 91: 1.0, 93: 1.0, 95: 1.0})]
[0.0
 SparseVector(100, {3: 0.0913, 4: 0.1448, 5: 0.1765, 10: 0.1429, 20: 0.006, 21: 0.006, 23: 1.0, 24: 1.0, 27: 1.0, 32: 0.0556, 40: 0.1, 53: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 71: 1.0, 73: 1.0, 80: 0.05, 81: 1.0, 83: 1.0, 85: 1.0, 87: 1.0, 91: 1.0, 93: 1.0})]
Tiempo de ejecución:  0.2047569751739502
None


In [289]:
n = 1
print("Row class:", data.head(n)[-1][0] ) # CLase/Label
prediction = predict_classification(total_array, total_array[0], k=3)
print('Expected label: %d, Got: %d.' % (total_array[0][0], prediction))

Row class: 0.0
[0.0, 0.0, 0.0]
0.0
Expected label: 0, Got: 0.


In [290]:
print(test_array[0][0])

0.0


In [ ]:
start_time = time.time()
k_nearest_neighbors(train_array, test_array, k = 3)
end_time = time.time()
print(tiempo(start_time, end_time))

In [267]:
print('RDD de entrenamiento: ' + str(rdd_train.count()))
print('RDD de test: ' + str(rdd_test.count()))

RDD de entrenamiento: 131
RDD de test: 69


In [268]:
# Metodo que guarda cada renglon en un archivo .svm
def save_file(data):
    file = open('../data/url_svmlight/Distancia_euclideana_100_x_500000.svm', 'a')
    file.write(data)
    file.close()

In [269]:
"""
[summary:
    Método para calcular la distancia euclídea entre cada una de las 
    columnas del conjunto de test respecto a las columnas del conjunto
    de entrenamiento.
]

Args:
    instance ([pyspark.sql.types.Row]): [
        Recibe cada una de las instancias que hay en el dataset
    ]
"""
def euclidean_distance(instance):
    distance = 0
    instance_distance = ''
    for row in range(len(train_array)):
        instance_distance += str(train_array_labels[row][0]) + ' '
        for column in range(len(instance[1])):
            distance = pow(train_array[row][0][column] - instance.features[column], 2)
            distance = math.sqrt(distance)
            # instance_distance += str(column + 1) +':' + str(distance) + ' ' # -> Si quisiera poner los indices de cada caracteristica.
            instance_distance += str(distance) + ' '
        instance_distance += '\n'
    save_file(instance_distance)

In [ ]:
# Ejecuta el método que calcula la distancia euclídea entre los puntos euclidean_distance()
test.foreach(euclidean_distance)

In [ ]:
rdd_samp1 = sc.textFile('../data/url_svmlight/arch_prb.svm')
rdd_samp2 = sc.textFile('../data/url_svmlight/arch_prb1.svm')
rdd_samp3 = sc.textFile('../data/url_svmlight/arch_prb2.svm')

In [ ]:
five_nearest1 = rdd_samp1.takeOrdered(5)
five_nearest2 = rdd_samp2.takeOrdered(5)
five_nearest3 = rdd_samp3.takeOrdered(5)

In [ ]:
def class_average(five_nearest):
    mean = 0
    for i in range(5):
        mean += float(five_nearest[i][0])
    mean = mean / 5
    if(mean > 0.5):
        print('Clase K-NN: 1')
        return 1
    else:
        print('Clase K-NN: 0')
        return 0

In [ ]:
def accuracy():
    lista = [five_nearest1, five_nearest2, five_nearest3]
    accuracy = 0.0
    for i in range(len(test_array_labels)):
        if(test_array_labels[i][0] == class_average(lista[i])):
            accuracy += 1
        print('Clase Real: ' + str(test_array_labels[i][0]))
        print('\n')
    accuracy = accuracy / len(test_array_labels)
    print('Accuracy: ' + str(accuracy))

In [ ]:
accuracy()

In [ ]:
lista = [five_nearest1, five_nearest2, five_nearest3]

In [ ]:
# listaclass_average

In [ ]:
# five_nearest[0][0] # Clase

In [ ]:
# rdd_prueba2 = sc.textFile('../data/url_svmlight/Distancia_euclideana_5_x_76.svm')

In [ ]:
five_nearest2 = rdd_prueba.takeOrdered(5)
type(five_nearest2)

In [ ]:
five_nearest2[0][0]

In [138]:
# Prueba de la función euclidean_distance(), se mandan dos renglones del dataset total

start  = time.time()
length = data.count()               # Se obtiene el total de renglones en el dataset
for row in range(1, (length + 1)):
    distance = euclidean_distance(data.head(1)[-1], data.head(row)[-1])
    #print("Dinstancia del renglon 1 con el", row, ":", distance)

end = time.time()

tiempo(start, end)

Tiempo de ejecución:  9.42785382270813


In [33]:
start = time.time()
data.head(175)[-1]
end = time.time()

tiempo(start, end)

Tiempo de ejecución:  0.03139948844909668


In [26]:
data.select('* where label=0')

AnalysisException: cannot resolve '`* where label=0`' given input columns: [features, label];
'Project ['* where label=0]
+- Relation[label#10,features#11] libsvm


In [53]:
data.head(1)[-1][1][0]

0.0

In [66]:
for i in range(76):
    print(data.head(10)[-1][1][i])
    #print(i)

0.0
1.0
0.0
0.0456432
0.0758621
0.0588235
0.0
0.0
1.0
1.0
0.142857
0.0
0.0
0.0
0.0
0.1
0.895527
0.839999
0.129503
0.0
0.00595074
0.00595238
0.00595238
1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0555556
1.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
